In [1]:
import duckdb
import pandas as pd

%load_ext sql
%pip install duckdb-engine --quiet
%sql duckdb://

Note: you may need to restart the kernel to use updated packages.


Connecting to 'duckdb://'

In [2]:
# duckdb.sql("CREATE TABLE users AS SELECT * FROM read_csv_auto('table_users.csv');")
# print(duckdb.sql("SELECT * FROM users LIMIT 10;"))

# duckdb.sql("CREATE TABLE user_brands AS SELECT * FROM read_csv_auto('table_user_brands.csv');")
# print(duckdb.sql("SELECT * FROM user_brands LIMIT 10;"))

In [3]:
%%sql 
CREATE TABLE users AS SELECT * FROM read_csv_auto('table_users.csv');
CREATE TABLE user_brands AS SELECT * FROM read_csv_auto('table_user_brands.csv');

Running query in 'duckdb://'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [4]:
%%sql
SELECT * FROM users LIMIT 5;

Running query in 'duckdb://'

user_id,first_name,last_name,email,favorite_color,birth_year,is_adult
0,Jodi,Peterson,jodi.peterson@att.net,Unmellow Yellow,1999,True
1,Jenna,Bond,jenna.bond@home.nl,Neon Carrot,1988,True
2,Harrison,Pope,harrison.pope@yahoo.in,Neon Carrot,1963,True
3,June,Copeland,june.copeland@voila.fr,Blizzard Blue,1970,True
4,Juanita,Meyer,juanita.meyer@cox.net,Hot Magenta,1967,True


In [5]:
%%sql
SELECT * FROM user_brands LIMIT 5;

Running query in 'duckdb://'

user_brand_id,user_id,brand_event_name
0,0,Office Depot
1,0,Williams
2,0,Toys `R` Us
3,0,Oracle
4,0,Tyson Foods


In [22]:
%%sql
WITH joined_data AS
    (
    SELECT  u.*, ub.*
    FROM    (SELECT * FROM users) AS u
            INNER JOIN
            (SELECT * FROM user_brands) as ub
    ON    
            u.user_id = ub.user_id
    ), 
    joined_counts AS
    (
    SELECT user_id, COUNT(brand_event_name) AS event_count
    FROM   joined_data
    GROUP BY user_id
    ORDER BY event_count
    )
SELECT 
    COUNT(CASE WHEN event_count >= 0  AND event_count <= 0   THEN 1 END) AS "[0  - 0 ]",
    COUNT(CASE WHEN event_count >= 1  AND event_count <= 5   THEN 1 END) AS "[1  - 5 ]",
    COUNT(CASE WHEN event_count >= 6  AND event_count <= 10  THEN 1 END) AS "[6  - 10]",
    COUNT(CASE WHEN event_count >= 11 AND event_count <= 20  THEN 1 END) AS "[11 - 20]",
    COUNT(CASE WHEN event_count >= 21 AND event_count <= 999 THEN 1 END) AS "[21 - + ]"
FROM joined_counts;

Running query in 'duckdb://'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[0 - 0 ],[1 - 5 ],[6 - 10],[11 - 20],[21 - + ]
0,21936136,9801761,1234188,21664


In [ ]:
%%sql
